# CSE 6240 - Web Search & Text Mining - Homework 3

In [1]:
# __author__ = 'Bhanu Verma'
# GTid = '903151012'

In [114]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg
import os, os.path
from scipy.sparse import lil_matrix,linalg
from scipy.sparse import csr_matrix
from sklearn.cross_validation import train_test_split
from scipy.stats import pearsonr
%matplotlib inline

In [30]:
def get_matrix_dimensions():
    doc_size = 0
    vocab_size = 0
    with open(filepath, "r") as f:
        count = 0
        for line in f:
            line = line.rstrip('\n')
            if count==0:
                doc_size = int(line)
            elif count==1:
                vocab_size = int(line)
            else:
                break
                
            count+=1
    
    f.close()
    
    return (doc_size,vocab_size)

In [31]:
def populate_matrix():
    with open(filepath, "r") as f:
        count = 0
        for line in f:
            line = line.rstrip('\n')
            if count > 2:
                arr = line.split()
                i = int(arr[0])-1
                j = int(arr[1])-1
                val = int(arr[2])
                mat[i,j] = val
            count+=1
    f.close()

In [107]:
def get_triplet_list():
    triplet_list = []
    ordered_list = []
    for i in range(n_doc):
        # randomly generate 6 numbers, 1 extra to take care of the case where random number id is equal to i
        rand_arr = np.random.choice(n_doc,6,replace=False)
        doc_list = [] # list of indices of documents
        corr_val = [] # list of calculated correlation coefficients
        tuple_list = []
        
        # fill up correlation values
        for j in range(len(rand_arr)):
            if i!=j:
                a = get_row(i)
                b = get_row(rand_arr[j])
                pear_val = pearsonr(a,b) # it returns a tuple where first element is the coefficient value
                corr_val.append(pear_val[0])
                doc_list.append(rand_arr[j])
                if len(corr_val) == 5:
                    break

        # build up triplets
        for a in range(len(corr_val)):
            for b in range(a+1,len(corr_val)):
                if b < len(corr_val):
                    if corr_val[a] >= corr_val[b]:
                        triplet_list.append((i,doc_list[a],doc_list[b]))
                        tuple_list.append((i,doc_list[a],doc_list[b]))
                    else:
                        triplet_list.append((i,doc_list[b],doc_list[a]))
                        tuple_list.append((i,doc_list[a],doc_list[b]))
        
    return triplet_list

In [109]:
def get_dense_row(index):
    return dense_mat[index].tolist()[0] # fix for the issue, tolist() here returns matrix of rowsize 1

In [112]:
def get_sparse_row(index):
    return mat[index]

# Question 2.a

In [85]:
# get matrix dimensions
filepath = 'Data/docword.kos.txt'
size_tup = get_matrix_dimensions()

In [84]:
n_doc = size_tup[0]
n_vocab = size_tup[1]

In [88]:
# populate lil_matrix dimensions
mat = lil_matrix((size_tup[0],size_tup[1]),dtype=np.int32)
populate_matrix()

# convert lil_matrix to csr matrix for faster computation
mat = mat.tocsr()
dense_mat = mat.todense()

In [110]:
triplets = get_triplet_list()

In [100]:
# split triplet array into training and testing data
train,test = train_test_split(triplets, train_size = 0.8)

# Question 1 - OASIS Implementation

In [106]:
w = np.identity(n_vocab)

In [151]:
def get_similarity_score(a,b):
    temp_result = a.dot(w)
    temp_result = csr_matrix(temp_result)
    result = temp_result.dot(b)
    return result[0,0]

In [152]:
def get_loss(p_i,p_plus,p_neg):
    res_a = get_similarity_score(p_i,p_plus.transpose())
    res_b = get_similarity_score(p_plus,p_neg.transpose())
    val = 1-res_a+res_b
    return max(0,val)

In [153]:
for i in range(1):
    p_i = mat[train[i][0]]
    p_plus = mat[train[i][1]]
    p_neg = mat[train[i][2]]
    loss = get_loss(p_i,p_plus,p_neg)
    

12.0
74.0
4.0
0
0
0
1.0
0
52.0
0
